In [1]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
from redfin import Redfin

client = Redfin()



In [63]:
address = '155 Manordale Dr'

response = client.search(address)
url = response['payload']['exactMatch']['url']
initial_info = client.initial_info(url)

property_id = initial_info['payload']['propertyId']
mls_data = client.below_the_fold(property_id)

listing_id = initial_info['payload']['listingId']
avm_details = client.avm_details(property_id, listing_id)
region_id = client.primary_region(url)

In [64]:
mls_data['payload']

{'amenitiesInfo': {'provider': '',
  'superGroups': [{'types': [21],
    'amenityGroups': [{'groupTitle': 'Bathroom Information',
      'referenceName': 'Bathroom Information',
      'amenityEntries': [{'amenityName': '# of Full Baths',
        'referenceName': 'FULL_BATHS',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['2']},
       {'amenityName': '# of 1/2 Baths',
        'referenceName': 'HALF_BATHS',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['1']}]}],
    'titleString': 'Interior'},
   {'types': [27, 20],
    'amenityGroups': [{'groupTitle': 'Exterior Information',
      'referenceName': 'Exterior Features',
      'amenityEntries': [{'amenityName': 'Building Style Type',
        'referenceName': 'STYLE_CODE',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['Townhouse']},
       {'amenityName': 'Building Type',
        'referenceName': 'BUILDING_CODE',
        'accessLevel': 1,
    

In [19]:
avm_details['payload']

{'listingTimezone': 'US/Eastern',
 'numBeds': 2,
 'numBaths': 2.5,
 'sqFt': {'displayLevel': 1, 'value': 1552},
 'listingPrice': 405000,
 'lastSoldPrice': 265720,
 'lastSoldDate': 1545984000000,
 'isServiced': True,
 'isActivish': True,
 'isHidden': False,
 'isVisibilityRestricted': False,
 'soldDate': 1545984000000,
 'soldDateTimeZone': 'US/Eastern',
 'latLong': {'latitude': 35.911644, 'longitude': -78.994254},
 'priceInfo': {'amount': 405000,
  'label': 'Price',
  'displayLevel': 1,
  'dataSourceId': 102},
 'searchStatusId': 1,
 'streetAddress': {'streetNumber': '155',
  'directionalPrefix': '',
  'streetName': 'Manordale',
  'streetType': 'Dr',
  'directionalSuffix': '',
  'unitType': '',
  'unitValue': '',
  'addressDisplayLevel': {'displayLevel': 1,
   'displayText': '',
   'dataSourceId': 102},
  'assembledAddress': '155 Manordale Dr',
  'includeStreetNumber': True,
  'includeUnitNumber': True,
  'includeStreetName': True},
 'dataSources': [],
 'historicalGraphEnabled': True,
 's

In [ ]:
land_sq_ft = mls_data['payload']['amenitiesInfo']['superGroups'][1]['amenityGroups'][3]['amenityEntries'][1]['amenityValues'][0]
parking_spaces =  mls_data['payload']['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][0]['amenityValues'][0]
parking_type = mls_data['payload']['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][1]['amenityValues'][0]
home_type = mls_data['payload']['amenitiesInfo']['superGroups'][1]['amenityGroups'][0]['amenityEntries'][3]['amenityValues'][0]

print("Land Sq. Ft:", land_sq_ft)
print("# of Parking Spaces:", parking_spaces)
print("Parking Type:", parking_type)
print("Home Type:", home_type)

In [ ]:
biggie = pd.DataFrame.from_dict(avm_details)
df = biggie.drop(["version", "errorMessage", "resultCode"], axis = 1)
df_t = df.T
df['payload']['streetAddress']['streetName']

In [ ]:
pr = client.primary_region(url)
pr

In [59]:
if client.rental_estimate(property_id, listing_id)['payload']['rentalEstimateInfo']['shouldShow'] == "True":
    print(1)
else:
    print(2)

2


In [ ]:
city = "Champaign"  # Specify the city name
state = "IL" #Give state
radius = 5  # Specify the radius in miles
property_type = "house"  # Specify the property type, e.g., "houses", "condos", "townhouses", etc.
max_price = 400000
# Perform the search
search_response = client.search(city)
#url = search_response['payload']['exactMatch']['url']
#sr = pd.DataFrame.from_dict(search_response)
search_response

In [ ]:
places = search_response['payload']['sections'][0]['rows']
for place in places:
    print("Name:", place['name'])
    print("Sub Name:", place['subName'])
    print("URL:", place['url'])
    print("Active:", place['active'])
    print("Internal Search Volume:", place['internalSearchVolume'])
    print("--------------------------------------")

In [ ]:
def process_html_snippet(html_snippet):
# Parse the HTML snippet using BeautifulSoup
    soup = BeautifulSoup(html_snippet, 'html.parser')

    # Find the div tag
    div_tag = soup.find('div', class_='link-and-anchor')
    # Extract the address from the div tag
    address = div_tag.text

    # Print the address
    return(address)

ht = '<a href="/CA/Tracy/224-E-12th-St-95376/unit-B/home/19762647" title="224 E 12th St Unit B, Tracy, CA 95376" target="_blank"><div class="link-and-anchor">224 E 12th St Unit B, Tracy, CA 95376</div></a>'
process_html_snippet(ht)

In [ ]:
def get_all_houses(purl):
    
    url = 'https://www.redfin.com' + purl
    print(url)
    # Make an HTTP GET request to fetch the HTML content
    response = requests.get(url)
    html_content = response.text

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all HomeCardsContainers
    home_cards_containers = soup.find_all('div', class_='v2 interactive')
    print(home_cards_containers)
    addresses = []
    
    # Iterate over each HomeCardsContainer
    for container in home_cards_containers:
        # Find the anchor tag within the HomeCardsContainer
        anchor_tag = container.find('a')

        # Extract the HTML snippet of the house
        html_snippet = str(anchor_tag)
        
        
        # Process the HTML snippet or perform further operations
        # For example, you can pass it to another function or store it in a list
        addy = process_html_snippet(html_snippet)
        
        addresses.append(addy)
        
    return addresses

purl = "/city/3210/IL/Champaign"

print(get_all_houses(purl))

In [47]:
add = '4544 Radnor Street'

res = client.search(add)
urll = res['payload']['exactMatch']['url']
ii = client.initial_info(urll)

p_id = ii['payload']['propertyId']
m_data = client.below_the_fold(p_id)

l_id = ii['payload']['listingId']
avm = client.avm_details(p_id, l_id)


m_data['payload']

{'amenitiesInfo': {'provider': '',
  'superGroups': [{'types': [21],
    'amenityGroups': [{'groupTitle': 'Basement Information',
      'referenceName': 'Basement Information',
      'amenityEntries': [{'amenityName': 'Basement Sq. Ft',
        'referenceName': 'BASEMENT_SQUARE_FEET',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['656']}]},
     {'groupTitle': 'Bathroom Information',
      'referenceName': 'Bathroom Information',
      'amenityEntries': [{'amenityName': '# of Full Baths',
        'referenceName': 'FULL_BATHS',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['1']}]},
     {'groupTitle': 'Heating & Cooling',
      'referenceName': 'Heating & Cooling',
      'amenityEntries': [{'amenityName': 'Heating Fuel Type',
        'referenceName': 'FUEL_CODE',
        'accessLevel': 1,
        'displayLevel': 1,
        'amenityValues': ['Gas']},
       {'amenityName': 'Heating Type',
        'referenceName': 'HEATING_

In [46]:
avm_details['payload']

{'listingTimezone': 'US/Eastern',
 'numBeds': 2,
 'numBaths': 2.5,
 'sqFt': {'displayLevel': 1, 'value': 1552},
 'listingPrice': 405000,
 'lastSoldPrice': 265720,
 'lastSoldDate': 1545984000000,
 'isServiced': True,
 'isActivish': True,
 'isHidden': False,
 'isVisibilityRestricted': False,
 'soldDate': 1545984000000,
 'soldDateTimeZone': 'US/Eastern',
 'latLong': {'latitude': 35.911644, 'longitude': -78.994254},
 'priceInfo': {'amount': 405000,
  'label': 'Price',
  'displayLevel': 1,
  'dataSourceId': 102},
 'searchStatusId': 1,
 'streetAddress': {'streetNumber': '155',
  'directionalPrefix': '',
  'streetName': 'Manordale',
  'streetType': 'Dr',
  'directionalSuffix': '',
  'unitType': '',
  'unitValue': '',
  'addressDisplayLevel': {'displayLevel': 1,
   'displayText': '',
   'dataSourceId': 102},
  'assembledAddress': '155 Manordale Dr',
  'includeStreetNumber': True,
  'includeUnitNumber': True,
  'includeStreetName': True},
 'dataSources': [],
 'historicalGraphEnabled': True,
 's

In [29]:
client.rental_estimate(property_id, listing_id)['payload']

{'rentalEstimateInfo': {'displayLevel': 1,
  'propertyId': 147232991,
  'predictedValue': 2078,
  'predictedValueLow': 1876,
  'predictedValueHigh': 2463,
  'shouldShow': True},
 'displayType': 3,
 'numBeds': 2,
 'propertyTypeId': 13,
 'streetAddress': {'streetNumber': '155',
  'directionalPrefix': '',
  'streetName': 'Manordale',
  'streetType': 'Dr',
  'directionalSuffix': '',
  'unitType': '',
  'unitValue': '',
  'addressDisplayLevel': {'displayLevel': 1,
   'displayText': '',
   'dataSourceId': 102},
  'assembledAddress': '155 Manordale Dr',
  'includeStreetNumber': True,
  'includeUnitNumber': True,
  'includeStreetName': True},
 'previewText': '$1,876 - $2,463 / mo'}

In [42]:
cur = client.info_panel(property_id, listing_id)['payload']

In [44]:
cur

{'mainHouseInfo': {'listingId': 168592085,
  'openHousesDisplayLevel': 1,
  'videoOpenHouses': [],
  'hotnessInfo': {'isHot': False,
   'hotnessMessageInfoWithTourLink': {'hotnessMessageAction': 'go tour it now'}},
  'source': {'dataSourceId': 102,
   'dataSourceDescription': 'Triangle Multiple Listing Service (TMLS)',
   'dataSourceName': 'TMLS',
   'shouldShowLargerLogo': False},
  'listingAgents': [{'agentInfo': {'agentName': 'Christina Chenet',
     'isAgentNameBlank': False,
     'isRedfinAgent': False,
     'isPartnerAgent': False,
     'isExternalAgent': False},
    'brokerName': 'Urban Durham Realty',
    'isOpendoor': False}],
  'buyingAgents': [],
  'lastCheckedDate': 1684875083053,
  'lastUpdatedDate': 1682684140929,
  'remarksDisplayLevel': 1,
  'marketingRemarks': [{'marketingRemark': "Tasteful finishes, sought-after community, convenient Durham/Chapel Hill location. .. this Chapel Run charmer has it all!  Built in 2018 by David Weekley Homes, this immaculate and like-new 